In [2]:
import os
import subprocess
import argparse
import re

### Extract eng subtitle using ffprobe

In [3]:
# test block
random = 'ffprobe -loglevel error -select_streams s -show_entries stream=index:stream_tags=language -of csv=p=0 video_file'
print(f'{random.split()}')

['ffprobe', '-loglevel', 'error', '-select_streams', 's', '-show_entries', 'stream=index:stream_tags=language', '-of', 'csv=p=0', 'video_file']


In [4]:
video_file = 'D:\Seeding\Movies\Prisoners 2013 1080p BluRay DTS x264-DON.mkv'

if not os.path.exists(video_file):
    print("Error: Video file not found")

command = [
    'ffprobe',
    '-loglevel', 'error',
    '-select_streams', 's',
    '-show_entries', 'stream=index:stream_tags=language',
    '-of', 'csv=p=0',
    f'{video_file}'
    ]

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
except subprocess.CalledProcessError as e:
    print("Error: ", e)

# print(result.stderr.split('\n'))
print(f'{result.stdout}')

2,por
3,bul
4,hrv
5,cze
6,dan
7,dut
8,eng
9,eng
10,est
11,fin
12,fre
13,ger
14,gre
15,hun
16,ita
17,nor
18,por
19,rum
20,rus
21,srp
22,slv
23,spa
24,swe
25,vie



### Parse ffprobe output to find English subtitle track

In [5]:
subtitle_tracks = []

# storing the first index no matter the language
for index, line in enumerate(result.stdout.split('\n')):
    if index == 0:
        first_sub_track_offset = line.split(',')[0]
        print(f'Setting the offset: {first_sub_track_offset}')
    if 'eng' in line:
        print(f'Found english sub track at index {line.split(",")[0]}')

        # removing the offset in order to get the true entry
        entry = int(line.split(",")[0]) - int(first_sub_track_offset)
        subtitle_tracks.append(entry)

Setting the offset: 2
Found english sub track at index 8
Found english sub track at index 9


In [6]:
if not subtitle_tracks:
    print("No English subtitle track found")
else:
    print(subtitle_tracks)

[6, 7]


### Extract subtitles from the first English subtitle track found

In [7]:
subtitle_track = subtitle_tracks[0]
filename, _ = os.path.splitext(video_file)
subtitle_file = f"{filename}.srt"

### Mock of the full ffmpeg command before actually runnning it

In [8]:
print(f'ffmpeg -i {video_file} -map 0:s:{subtitle_track} {subtitle_file}')

ffmpeg -i D:\Seeding\Movies\Prisoners 2013 1080p BluRay DTS x264-DON.mkv -map 0:s:6 D:\Seeding\Movies\Prisoners 2013 1080p BluRay DTS x264-DON.srt


### Execute FFmpeg command to extract English subtitles

In [9]:
command = [
    'ffmpeg', '-i', video_file,
    '-vn', '-an',
    '-map', f'0:s:{subtitle_track}',
    subtitle_file]

try:
    subprocess.run(['ffmpeg', '-i', video_file, '-map', f'0:s:{subtitle_track}', subtitle_file], check=True)
    print("English subtitles extracted successfully")
except subprocess.CalledProcessError as e:
    print(f'Error on extracting subtitle: ', e)

English subtitles extracted successfully
